In [48]:
##-----------------------------------------##
#               Packages
##-----------------------------------------##
from datetime import datetime as dt
import dash
import dash_html_components as html
import dash_core_components as dcc

from IPython.display import Image
import ee, datetime
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import plotly.express as px


# Earth Engine Python API
ee.Initialize()

##-----------------------------------------##
#               Start Dash Date Range
##-----------------------------------------##



external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.layout = html.Div([
    dcc.DatePickerRange(
        id='my-date-picker-range',
        min_date_allowed=dt(1995, 8, 5),
        max_date_allowed=dt(2017, 9, 19),
        initial_visible_month=dt(2017, 8, 5),
        end_date=dt(2017, 8, 25)
    ),
    html.Div(id='output-container-date-picker-range')
])


@app.callback(
    dash.dependencies.Output('output-container-date-picker-range', 'children'),
    [dash.dependencies.Input('my-date-picker-range', 'start_date'),
     dash.dependencies.Input('my-date-picker-range', 'end_date')])

def update_output(start_date, end_date):
    string_prefix = 'You have selected: '
    
    if start_date is not None:
        start_date = dt.strptime(start_date.split('T')[0], '%Y-%m-%d')
        start_date_string = start_date.strftime('"%Y-%m-%d"')
        string_prefix = string_prefix + 'Start Date: ' + start_date_string + ' | '
        eeStart = ee.Date(start_date_string)
        
    if end_date is not None:
        end_date = dt.strptime(end_date.split('T')[0], "%Y-%m-%d")
        end_date_string = end_date.strftime('"%Y-%m-%d"')
        string_prefix = string_prefix + 'End Date: ' + end_date_string
        eeEnd = ee.Date(end_date_string)
    
    if len(string_prefix) == len('You have selected: '):
        return 'Select a date to see it displayed here'
    else:
        #return string_prefix
        #return ('"{}","{}"'.format(start_date_string, end_date_string))
        #print ('"{}","{}"'.format(start_date_string, end_date_string))
        
        #date = ('{}, {}'.format(start_date_string, end_date_string))
        date = ('{}, {}'.format(eeStart, eeEnd))
        #return ('{}, {}'.format(start_date_string, end_date_string))
        #print ('{}, {}'.format(start_date_string, end_date_string))
        return date

##-----------------------------------------##
#               GEE Code
##-----------------------------------------##

point = {'type':'Point', 'coordinates':[48.510648, -13.517244]}; # This point is on turbid water

# Getting ROI through array of coordinates (point of polygon)
poly = ee.Geometry.Polygon([[
    [48.462066650390625, -13.570240350544672],
    [48.546009063720696, -13.570240350544672],
    [48.546009063720696, -13.497140448129723],
    [48.462066650390625, -13.497140448129723],
    [48.462066650390625, -13.570240350544672]
]])



def makeLandsatSeries(date):
    lt4 = ee.ImageCollection('LANDSAT/LT04/C01/T1_SR')
    lt5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR')
    le7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR')
    lc8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')

    lt4 = lt4.select(['B1','B2','B3','B4','B5','B7','pixel_qa'],['blu','grn','red','nir','swir1','swir2','pixel_qa'])
    lt5 = lt5.select(['B1','B2','B3','B4','B5','B7','pixel_qa'],['blu','grn','red','nir','swir1','swir2','pixel_qa'])
    le7 = le7.select(['B1','B2','B3','B4','B5','B7','pixel_qa'],['blu','grn','red','nir','swir1','swir2','pixel_qa'])
    lc8 = lc8.select(['B2','B3','B4','B5','B6','B7','pixel_qa'],['blu','grn','red','nir','swir1','swir2','pixel_qa'])

    fullCollection = ee.ImageCollection(lt4.merge(lt5).merge(le7).merge(lc8))

    region = poly

    filteredCollection = fullCollection.filterDate(date).filterBounds(region)                
    return filteredCollection 
    print (filteredCollection)


fullcollection = makeLandsatSeries(date)  # This is an ee.ImageCollection


# Get Info which is the base for dataframe & Column Names from Info
info = fullcollection.getRegion(point,200).getInfo() # 200meters buffer

header = info[0]
#print(header) #['id', 'longitude', 'latitude', 'time', 'blu', 'grn', 'red', 'nir', 'swir1', 'swir2', 'pixel_qa']

# Create dataframe using data from info & columns from info[0]
df = pd.DataFrame(info,columns = ['id', 'longitude', 'latitude', 'time', 'blu', 'grn', 'red', 'nir', 'swir1', 'swir2', 'pixel_qa'])

# Create an ID column with NAN values
df['Satellite_ID'] = ('NAN')

# Make sure that all NaN values are `np.nan` not `'NAN'` (strings)
df = df.replace('NAN', np.nan)

# ------------------------------------------------------------------------ #
# ------------------------------------------------------------------------ #

#            Search for specific string within the id column
# Once you have mask the idea is simple. Write out desired text say L4 into a wherever mask is True

#                         Note About na=True 
# mask = str.contains(..., na=False) or =True is used to make the NaNs be treated as True (or False)
# In this case we are overwriting all the NAN cells with all possible satellite ID, so we dont need it

# ------------------------------------------------------------------------ #
# ------------------------------------------------------------------------ #

mask = df['id'].str.contains(r'LT04')
df.loc[mask, 'Satellite_ID'] = ('L4')

mask = df['id'].str.contains(r'LT05')
df.loc[mask, 'Satellite_ID'] = ('L5')

mask = df['id'].str.contains(r'LT05')
df.loc[mask, 'Satellite_ID'] = ('L5')

mask = df['id'].str.contains(r'LE07')
df.loc[mask, 'Satellite_ID'] = ('L7')

mask = df['id'].str.contains(r'LC08')
df.loc[mask, 'Satellite_ID'] = ('L8')

# Drop First row (a duplicate of column header)
df = df.drop(df.index[0])

# The Earth Engine time stamp in milliseconds since the UNIX epoch.
# Link GEE: https://developers.google.com/earth-engine/glossary

# Convert UNIX to datetime
# Use pd.to_datetime() to convert unix epoch time
# Result is like this: 1994-03-17 06:11:20.254
df['time_stamp'] = pd.to_datetime(df['time'], unit='ms')

# Convert to String inorder to apply string split
df['time_string'] = df['time_stamp'].astype(str)

# make the new date column using string indexing [0:10] will give us the year,month,date part
# Result is like this 1994-03-17
df['date'] = df['time_string'].str[0:10]

# Convert back to datetime
df['date'] = pd.to_datetime(df['date'])

# Drop unnecessary columns
#df = df.drop('time_stamp', axis=1)
df = df.drop('time_string', axis=1)

# Calculate NDWI
#ndwi = (green - swir) / (green + swir)
df['NDWI'] = (df['grn'] - df['swir1']) / (df['grn'] + df['swir1'])
df['NDWI'] = pd.to_numeric(df['NDWI'])

# Simplified Table for Dash

dash_df = df[['Satellite_ID', 'longitude', 'latitude', 'NDWI', 'date']].copy()
dash_df

##-----------------------------------------##
#               GEE Code End
##-----------------------------------------##

if __name__ == '__main__':
    app.run_server(debug=True)

EEException: DateRange: Cannot interpret [1995-8-5, 1995-9-5] as a Date.

In [52]:
def update_output(start_date, end_date):
    string_prefix = 'You have selected: '
    
    if start_date is not None:
        start_date = dt.strptime(start_date.split('T')[0], '%Y-%m-%d')
        start_date_string = start_date.strftime('"%Y-%m-%d"')
        string_prefix = string_prefix + 'Start Date: ' + start_date_string + ' | '
        eeStart = ee.Date(start_date_string)
        
    if end_date is not None:
        end_date = dt.strptime(end_date.split('T')[0], "%Y-%m-%d")
        end_date_string = end_date.strftime('"%Y-%m-%d"')
        string_prefix = string_prefix + 'End Date: ' + end_date_string
        eeEnd = ee.Date(end_date_string)
    
    if len(string_prefix) == len('You have selected: '):
        return 'Select a date to see it displayed here'
    else:
        date = ('{}, {}'.format(eeStart, eeEnd))
        return date
print(type(date))

<class 'tuple'>


In [53]:
update_output("1984-01-01", "1984-01-01")

'ee.Date({\n  "type": "Invocation",\n  "arguments": {\n    "value": "\\"1984-01-01\\""\n  },\n  "functionName": "Date"\n}), ee.Date({\n  "type": "Invocation",\n  "arguments": {\n    "value": "\\"1984-01-01\\""\n  },\n  "functionName": "Date"\n})'